In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
import lime.lime_tabular
import numpy as np
from IPython.display import IFrame
import shap
import lime
import lime.lime_tabular

In [ ]:
df = pd.read_csv('/kaggle/input/airline/AirlineScrappedReview_withSentiment.csv')
traveller_type_map = {'Solo Leisure': 0, 'Couple Leisure': 1, 'Family Leisure': 2, 'Business': 3, 'Unknown': 4, 'Various': 5}
df['Traveller_Type'] = df['Traveller_Type'].map(traveller_type_map)

class_map = {'Economy Class': 0, 'Business Class': 1, 'First Class': 2, 'Premium Economy': 3, 'unkown': 4}
df['Class'] = df['Class'].map(class_map)

verified_map = {'Trip Verified': 0, 'Not Verified': 1}
df['Verified'] = df['Verified'].map(verified_map)

X = df[['Traveller_Type', 'Class', 'Sentiment_score', 'Verified']]
y = df['satisfaction']

categorical_cols = ['Traveller_Type', 'Class']
numeric_cols = ['Sentiment_score', 'Verified']
X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)

FileNotFoundError: [Errno 2] No such file or directory: 'AirlineScrappedReview_withSentiment.csv'

In [ ]:
tree = DecisionTreeClassifier(
    max_depth=2,
    min_samples_split=20,
    random_state=42
)
tree.fit(X_train, y_train)

print(f"Decision Tree Accuracy on test set: {tree.score(X_test, y_test):.2f}")

plt.figure(figsize=(12, 6))
plot_tree(
    tree,
    feature_names=X_train.columns,
    class_names=['Not Satisfied', 'Satisfied'],
    filled=True,
    rounded=True,
    fontsize=10
)
plt.title("Decision Tree Visualization (Global Explanation)")
plt.show()

In [ ]:
import shap
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

explainer = shap.TreeExplainer(clf)
shap_values = explainer.shap_values(X_test)

if isinstance(shap_values, list):
    shap_class1 = shap_values[1]
else:
    shap_class1 = shap_values

shap.summary_plot(shap_class1, X_test, feature_names=X_test.columns)

In [ ]:
import shap
import numpy as np

X_fixed = X_train.copy().fillna(0)
X_test_fixed = X_test[X_fixed.columns].fillna(0)

background = X_fixed.sample(10, random_state=42)
X_instance = X_test_fixed.iloc[[0]]  # single instance

explainer = shap.KernelExplainer(clf.predict_proba, background)
shap_values = explainer.shap_values(X_instance)

shap.initjs()
shap.force_plot(
    explainer.expected_value[1], 
    shap_values[1], 
    X_instance,
    matplotlib=True
)

In [ ]:
import lime.lime_tabular
import numpy as np
from IPython.core.display import display, HTML

features_used = clf.feature_names_in_  # columns model was trained on

X_numeric = X_train[features_used].copy()
X_numeric = X_numeric.fillna(0)
X_test_numeric = X_test[features_used].copy().fillna(0)

explainer_lime = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_numeric.values,
    feature_names=list(X_numeric.columns),
    class_names=['Not Satisfied', 'Satisfied'],
    mode='classification',
    discretize_continuous=True
)

i = 0
instance = X_test_numeric.values[i]
exp = explainer_lime.explain_instance(
    instance,
    clf.predict_proba,
    num_features=len(features_used)
)

html_output = exp.as_html()
display(HTML(html_output))